In [3]:
import xarray as xr
import pandas as pd
import numpy as np
import multiprocessing
import os
import glob

In [22]:
# HDV file

out = pd.read_excel("atlantic/1902380_hdv.xlsx")

In [27]:
# PFT file
pft=xr.open_dataset("https://my.cmems-du.eu/thredds/dodsC/cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D")

In [24]:
# Function for parallelization
def parallelize(func, df):
    num_cores = multiprocessing.cpu_count()-1  # leave one free to not freeze machine
    df_split = np.array_split(df, num_cores) # split dataframe into chunks
    pool = multiprocessing.Pool(num_cores) # number of pool
    df = pd.concat(pool.map(func, df_split)) # concatenate result
    pool.close()
    pool.join()
    return df

In [21]:
# Exact date
# List of variable to mine
var_list = ['CHL','DIATO','HAPTO','GREEN','PROKAR','PROCHLO','MICRO','NANO','PICO']
for i in var_list:
    out[i] = out['Cycle']

def PFT_mining(df):
    for index, row in df.iterrows():
        temp = pft.sel(time=row['Date'],lat=row['Lat'],lon=row['Lon'],method="nearest")
        for i in var_list:
            df.at[index,i] = temp[i].values
    return df



NameError: name 'out' is not defined

In [ ]:
# Rolling 3 day, 1 core run
out['CHL'] = out['Cycle']
out['DIATO'] = out['Cycle']
out['DINO'] = out['Cycle']
out['HAPTO'] = out['Cycle']
out['GREEN'] = out['Cycle']
out['PROKAR'] = out['Cycle']
out['PROCHLO'] = out['Cycle']
out['MICRO'] = out['Cycle']
out['NANO'] = out['Cycle']
out['PICO'] = out['Cycle']

for index, row in out.iterrows():
    date_range = slice(row['Date']-np.timedelta64(24,'h'),row['Date']+np.timedelta64(24,'h'))
    temp = pft.sel(lat=row['Lat'],lon=row['Lon'],method="nearest").sel(time=date_range)
    mean = temp.mean('time')
    out.at[index,'CHL'] = mean.CHL.values
    out.at[index,'DIATO'] = mean.DIATO.values
    out.at[index,'DINO'] = mean.DINO.values
    out.at[index,'HAPTO'] = mean.HAPTO.values
    out.at[index,'GREEN'] = mean.GREEN.values
    out.at[index,'PROKAR'] = mean.PROKAR.values
    out.at[index,'PROCHLO'] = mean.PROCHLO.values
    out.at[index,'MICRO'] = mean.MICRO.values
    out.at[index,'NANO'] = mean.NANO.values
    out.at[index,'PICO'] = mean.PICO.values
out

In [25]:
# Multi-core run
# Function to mine data
def PFT_mining(df):
    for index, row in df.iterrows():
        date_range = slice(row['Date']-np.timedelta64(24,'h'),row['Date']+np.timedelta64(24,'h'))
        temp = pft.sel(lat=row['Lat'],lon=row['Lon'],method="nearest").sel(time=date_range)
        mean = temp.mean('time')
        for i in var_list:
            df.at[index,i] = mean[i].values
    return df



In [28]:
# List of variable to mine
var_list = ['CHL','DIATO','HAPTO','GREEN','PROKAR','PROCHLO','MICRO','NANO','PICO']
for i in var_list:
    out[i] = out['Cycle']
        
result = parallelize(PFT_mining,out)

In [31]:
out['Float'][1]

1902380

In [21]:
result.to_excel('5906339_hdv_PFT.xlsx')

In [34]:
# PFT file
pft=xr.open_dataset("https://my.cmems-du.eu/thredds/dodsC/cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D")
#filedir = os.path.expanduser('~')+"/OneDrive/Bioinformatics/argo/atlantic/"
filenames = glob.glob("atlantic/*.xlsx")
for file in filenames:
    out = pd.read_excel(file)
    # List of variable to mine
    var_list = ['CHL','DIATO','HAPTO','GREEN','PROKAR','PROCHLO','MICRO','NANO','PICO']
    for i in var_list:
        out[i] = out['Cycle']
    result = parallelize(PFT_mining,out)
    name = result['Float'][1]
    result.to_excel(f'{name}_hdv_PFT.xlsx')
    